In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import re
from time import sleep
from random import random
from time import time
import glob
import requests

In [2]:
#Save Credentials
with open('SpotifyID.txt', 'r') as f:
    client_id = f.read().replace('\n','')
with open('SpotifySecret.txt', 'r') as f:
    client_secret = f.read().replace('\n','')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [3]:
def get_token(client_seccret,client_id):
    AUTH_URL = 'https://accounts.spotify.com/api/token'

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']
    
    return access_token

In [4]:
def get_audio_analysis(track_id,access_token):
    
    headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
    }

    track_id = track_id.split(":")[-1]
    # base URL of all Spotify API endpoints
    BASE_URL = 'https://api.spotify.com/v1/'
    
    # actual GET request with proper header
    return(requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers).json())

In [5]:
def query(data_cleaned,file_loc):
    count=0
    uris, dates, popularities, audio_analysis=[],[],[],[]
    access_token = get_token(client_secret,client_id)
    for i in range(0,len(data_cleaned)):  
        #query artist and song
        artist=str(data_cleaned.loc[i,"artist"])
        title=str(data_cleaned.loc[i,"song"])

        #some songs have many artists associated
        #however spotify only lists few, so break the string into parts 
        #and search till match is found
        artist.replace(","," ")
        artist_string = [artist] + artist.strip().split()
        for artist in artist_string:
            if len(artist)>2:
                search_query = str(title) + ' ' + str(artist)
                result = sp.search(search_query)
                if result["tracks"]["items"]!=[]:
                    break
        
        if result["tracks"]["items"]==[]:
            uris.append("error")
            dates.append("error")
            popularities.append("error")
            audio_analysis.append({"error":""})   
            print("error",count,title,artist_string)
            count+=1
        
        else:
            print(i,result["tracks"]["items"][0]["uri"],artist)
            audio = get_audio_analysis(result["tracks"]["items"][0]["uri"],access_token)
            uri=result["tracks"]["items"][0]["uri"]
            date= result["tracks"]["items"][0]["album"]["release_date"]
            popularity = result["tracks"]["items"][0]["popularity"]  

            uris.append(uri)
            dates.append(date)
            popularities.append(popularity)
            audio_analysis.append(audio)

    return uris, dates, popularities, audio_analysis

In [6]:
def create_csv(uris, dates, popularities, audio_analysis, data_cleaned, file_loc):
    data_cleaned["uris"] = uris
    data_cleaned["popularity"] = popularities
    data_cleaned["date"] = dates
    
    
    temp = pd.DataFrame({"audio":audio_analysis})
    audio_analysis = pd.json_normalize(temp["audio"])
    data_cleaned = pd.concat([data_cleaned,audio_analysis],axis =1)
    data_cleaned.to_csv(file_loc)

In [ ]:
#read in songs from main df
datasets = glob.glob("./songs_wikipedia/*.csv")
datasets2 = glob.glob("./songs_wikipedia_final/*.csv")
for file_loc in datasets:
    data_cleaned = pd.read_csv(file_loc)
    for column in data_cleaned.columns:
        #data_cleaned[column] = data_cleaned[column].str.strip()
        data_cleaned = data_cleaned.dropna(subset =["song"]).reset_index(drop =True)
    file_loc = file_loc.replace("songs_wikipedia","songs_wikipedia_final")
    if file_loc not in datasets2:
        print(file_loc)
        uris, dates, popularities, audio_analysis = query(data_cleaned,file_loc)
        create_csv(uris, dates, popularities, audio_analysis, data_cleaned, file_loc)


In [13]:
datasets = glob.glob("./songs_lyrics/*.csv")
datasets2 = glob.glob("./songs_lyrics_final/*.csv")
for file_loc in datasets:
    data_cleaned = pd.read_csv(file_loc)
    for column in data_cleaned.columns:
        #data_cleaned[column] = data_cleaned[column].str.strip()
        data_cleaned = data_cleaned.dropna(subset =["song"]).reset_index(drop =True)
    file_loc = file_loc.replace("songs_lyrics","songs_lyrics_final")
    if file_loc not in datasets2:
        print(file_loc)
        uris, dates, popularities, audio_analysis = query(data_cleaned,file_loc)
        create_csv(uris, dates, popularities, audio_analysis, data_cleaned, file_loc)

In [ ]:
pd.read_csv("./songs_lyrics/Georgia.csv")

In [10]:
data_cleaned = pd.read_csv("./songs_lyrics/Alabama.csv")
for column in data_cleaned.columns:
    #data_cleaned[column] = data_cleaned[column].str.strip()
    data_cleaned = data_cleaned.dropna(subset =["song"]).reset_index(drop =True)
uris, dates, popularities, audio_analysis = query(data_cleaned,"./songs_lyrics_final/ Alabama.csv")
create_csv(uris, dates, popularities, audio_analysis, data_cleaned, "./songs_lyrics_final/ Alabama.csv")